In [86]:
import scipy.stats as scs
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import date, datetime, timedelta
from collections import Counter
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')
%matplotlib inline

In [87]:
import os

In [88]:
os.listdir('data')

['data_with_dummies.csv',
 'data_with_usd.csv',
 'data_with_dummies_and_class.csv',
 'data_nobadrows.csv',
 'all_data.csv',
 'data_with_pos',
 'ks-projects-201801.csv',
 'ks-projects-201612.csv',
 'data_with_pos.csv',
 'exchange.csv']

In [89]:
df = pd.read_csv('data/data_with_dummies_and_class.csv')

In [90]:
keep_cols = [x for x in df.columns if 'Unnamed' not in x]
df = df[keep_cols].set_index('ID')

In [91]:
df.shape

(694192, 250)

In [92]:
df.sample(5)

,backers,goal,pledged,usd pledged,usd_pledged_real,name$word_count,name$punc_count,name$caps_count,name$possesive_count,name$noun_count,...,launched_month&Jul,launched_month&Feb,launched_month&Dec,launched_month&Apr,launched_month&Sep,launched_month&Oct,launched_month&May,launched_month&Nov,launched_month&Jun,success
ID,,,,,,,,,,,,,,,,,,,,,
1272859096,227,10000,10647,1980.00000,10647.0,9,0.021277,0.170213,0.0,0.777778,...,False,False,False,False,True,False,False,False,False,True
1007121454,2,20000,6,6.00000,6.0,10,0.040816,0.122449,0.0,0.400000,...,False,False,False,False,False,False,False,False,False,False
1431471945,1,2500,15,0.00000,15.0,9,0.021277,0.127660,0.0,0.555556,...,False,False,False,False,False,False,False,False,False,False
1064353512,7,15000,127,134.45428,NaN,3,0.000000,0.142857,0.0,0.666667,...,False,False,True,False,False,False,False,False,False,False
1065478972,8,5000,618,618.00000,NaN,2,0.000000,0.200000,0.0,0.000000,...,False,False,False,False,True,False,False,False,False,False


In [93]:
features = Counter()
for col in df.columns:
    features[col.split('&')[0].strip()] += 1
features = pd.Series(features)

In [94]:
features

backers                      1
category                   159
country                     22
currency                    14
deadline_month              12
goal                         1
launched_month              12
length                       1
main_category               15
name$adj_count               1
name$caps_count              1
name$determinator_count      1
name$noun_count              1
name$possesive_count         1
name$preposition_count       1
name$punc_count              1
name$verb_count              1
name$word_count              1
pledged                      1
success                      1
usd pledged                  1
usd_pledged_real             1
dtype: int64

In [95]:
dfl = df.drop(['usd pledged', 'pledged'], axis=1)

In [96]:
df_logis = dfl.drop([col for col in dfl.columns if col.split('&')[0] == 'category'], axis=1)

In [97]:
df_logis = df_logis.dropna()

In [98]:
from sklearn import linear_model as lm

In [99]:
model = lm.LogisticRegression(C=.8)

In [100]:
cv = pd.Series(np.random.choice(range(5), df_logis.shape[0]), index=df_logis.index)

In [101]:
df_logis.index.shape

(374863,)

In [102]:
df_logis.index.unique().shape

(374863,)

In [103]:
Xtr = df_logis.loc[cv != 0, :]
Xte = df_logis.loc[cv == 0, :]
ytr = Xtr.pop('success')
yte = Xte.pop('success')

In [104]:
model.fit(Xtr, Xte)

ValueError: bad input shape (75383, 88)

In [ ]:
Xtr[np.isnan(Xtr['name$punc_count'])]

In [78]:
Xtr[Xtr['name$punc_count'] == np.inf]

,goal,backers,name$word_count,name$punc_count,name$caps_count,name$possesive_count,name$noun_count,name$adj_count,name$verb_count,name$preposition_count,...,launched_month&Mar,launched_month&Jul,launched_month&Feb,launched_month&Dec,launched_month&Apr,launched_month&Sep,launched_month&Oct,launched_month&May,launched_month&Nov,launched_month&Jun
ID,,,,,,,,,,,,,,,,,,,,,
518903190,3500,84,0,inf,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,False,False,False,False,False,False,False
